In [17]:
# Manually add the root path before running the test
import sys, os
sys.path.append(os.path.dirname(os.getcwd()))
from utils.helpers import (
    comparator_less,
    prepare_ancilla_cell_validity,
    Indexer
)


In [18]:
from math import ceil, log2
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit_aer import AerSimulator

from utils.helpers import Indexer
from oracle.column_uniqueness import column_uniqueness_circuit
from oracle.row_uniqueness import row_uniqueness_circuit
from oracle.cell_validity import cell_validity_circuit

# --- 1) PARAMETERS & GRID ---
n = 1
m = 3
# every row has duplicates -> row_constraints_flag should return 1
grid = [
    [None, None, 2]    
]

# --- 2) SET UP INDEXER + QUBIT POOL ---
# We need:
#  • data: n*n*k
#  • row_flags: n
#  • row_constraints_flag: 1
#  • comparator ancillas: k+1
k = max(1, int(ceil(log2(max(n,m)))))
num_anc = max(k + 2 + n + m, k+2*m+n-1, k+2*n+m-1)
idx = Indexer(grid, num_anc)

# One flat register for everything
qr = QuantumRegister(idx.total_qubits, name="q")
#cr = ClassicalRegister(1, name="c")   # measure only the final constraint flag
qc = QuantumCircuit(qr, name="cell_validity_demo")

idx.initialize_grid(qc)

# --- 4) APPLY THE ORACLE ---
row_uniqueness_circuit(qc, qr, idx)
column_uniqueness_circuit(qc, qr, idx)
cell_validity_circuit(qc, qr, idx, max(n,m))

# --- 5) COLLAPSE INTO GLOBAL FLAG ---
controls = [
    qr[idx.cell_valid_flag()],
    qr[idx.row_flag()],
    qr[idx.col_flag()]
]
global_q = qr[idx.global_flag()]
qc.mcx(controls, global_q, ctrl_state='0'*len(controls))
qc.x(global_q)

# --- 6) MEASURE 
qc.measure_all()

# --- 7) SIMULATE & PRINT RESULTS ---
sim = AerSimulator(method="matrix_product_state")
tcirc = transpile(qc, sim, optimization_level=3)
counts = sim.run(tcirc, shots=1).result().get_counts()

print("Total qubits:", qc.num_qubits)
print("Counts for col_constraints_flag:", counts)


Total qubits: 18
Counts for col_constraints_flag: {'000000000000100100': 1}


In [19]:
# --- after you’ve run and obtained `result.get_counts()` ---
bitstring = next(iter(counts))            # e.g. '01000101…'
# Qiskit’s bitstrings are MSB→LSB (left→right), qubit 0 is the rightmost bit
bits      = list(map(int, bitstring[::-1]))  # now bits[i] is the value of qubit i

n, m, k = idx.n, idx.m, idx.k

# 1) print out each qubit’s role and its measured value
print("Measured qubits:")
for q in range(idx.total_qubits):
    role = idx.pretty(q)
    print(f" q[{q:2d}] = {bits[q]}   ⟶ {role}")

# 2) reconstruct the grid from the data‐region into a 2D list
recovered = [[None for _ in range(m)] for _ in range(n)]
for i in range(n):
    for j in range(m):
        v = 0
        for b in range(k):
            bitval = bits[idx.data(i, j, b)]
            v |= (bitval << b)
        recovered[i][j] = v

# 3) print the grid row by row
print("\nReconstructed grid values:")
for i, row in enumerate(recovered):
    print(f" row {i:2d}: {row}")

# 4) check the final row‐constraint flag
rf = idx.global_flag()
print(f"\nGlobalflag (q[{rf}]) = {bits[rf]}")


Measured qubits:
 q[ 0] = 0   ⟶ data(0,0,0)
 q[ 1] = 0   ⟶ data(0,0,1)
 q[ 2] = 1   ⟶ data(0,1,0)
 q[ 3] = 0   ⟶ data(0,1,1)
 q[ 4] = 0   ⟶ data(0,2,0)
 q[ 5] = 1   ⟶ data(0,2,1)
 q[ 6] = 0   ⟶ row_flag
 q[ 7] = 0   ⟶ col_flag
 q[ 8] = 0   ⟶ cell_valid_flag
 q[ 9] = 0   ⟶ global_flag
 q[10] = 0   ⟶ ancilla(0)
 q[11] = 0   ⟶ ancilla(1)
 q[12] = 0   ⟶ ancilla(2)
 q[13] = 0   ⟶ ancilla(3)
 q[14] = 0   ⟶ ancilla(4)
 q[15] = 0   ⟶ ancilla(5)
 q[16] = 0   ⟶ ancilla(6)
 q[17] = 0   ⟶ ancilla(7)

Reconstructed grid values:
 row  0: [0, 1, 2]

Globalflag (q[9]) = 0
